# Bias-variance lab

In this lab you'll explore how bias and variance changes using a dataset on college statistics.

---

In [1]:
import numpy as np
import scipy 
import seaborn as sns
import pandas as pd
import patsy

from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso, RidgeCV, LassoCV
from sklearn.cross_validation import cross_val_score, KFold, train_test_split

import matplotlib
import matplotlib.pyplot as plt

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

plt.style.use('fivethirtyeight')


---

### Load data

Feel free to choose a target variable on your own. I chose "Grad.Rate" as my target variable but it's not required.

You'll want to discard the name of the college, and if you're planning on using the "Private" variable it will have to be changed into 1s and 0s rather than yes/no.

In [2]:
college = pd.read_csv('/Users/kiefer/github-repos/DSI-SF-2/datasets/college_stats/College.csv')

In [3]:
college.dtypes

Unnamed: 0      object
Private         object
Apps             int64
Accept           int64
Enroll           int64
Top10perc        int64
Top25perc        int64
F.Undergrad      int64
P.Undergrad      int64
Outstate         int64
Room.Board       int64
Books            int64
Personal         int64
PhD              int64
Terminal         int64
S.F.Ratio      float64
perc.alumni      int64
Expend           int64
Grad.Rate        int64
dtype: object

In [4]:
college.rename(columns={'Unnamed: 0':'College'}, inplace=True)

In [5]:
college.head(3)

,College,Private,Apps,Accept,Enroll,Top10perc,Top25perc,F.Undergrad,P.Undergrad,Outstate,Room.Board,Books,Personal,PhD,Terminal,S.F.Ratio,perc.alumni,Expend,Grad.Rate
0,Abilene Christian University,Yes,1660,1232,721,23,52,2885,537,7440,3300,450,2200,70,78,18.1,12,7041,60
1,Adelphi University,Yes,2186,1924,512,16,29,2683,1227,12280,6450,750,1500,29,30,12.2,16,10527,56
2,Adrian College,Yes,1428,1097,336,22,50,1036,99,11250,3750,400,1165,53,66,12.9,30,8735,54


In [6]:
gradrate = college['Grad.Rate'].values
X = college.iloc[:,1:-1]

In [7]:
X.head(2)

,Private,Apps,Accept,Enroll,Top10perc,Top25perc,F.Undergrad,P.Undergrad,Outstate,Room.Board,Books,Personal,PhD,Terminal,S.F.Ratio,perc.alumni,Expend
0,Yes,1660,1232,721,23,52,2885,537,7440,3300,450,2200,70,78,18.1,12,7041
1,Yes,2186,1924,512,16,29,2683,1227,12280,6450,750,1500,29,30,12.2,16,10527


In [8]:
X.Private = X.Private.map(lambda x: 1 if x == 'Yes' else 0)

In [9]:
X.columns = [x.lower().replace('.','_') for x in X.columns]

In [10]:
X.head(2)

,private,apps,accept,enroll,top10perc,top25perc,f_undergrad,p_undergrad,outstate,room_board,books,personal,phd,terminal,s_f_ratio,perc_alumni,expend
0,1,1660,1232,721,23,52,2885,537,7440,3300,450,2200,70,78,18.1,12,7041
1,1,2186,1924,512,16,29,2683,1227,12280,6450,750,1500,29,30,12.2,16,10527


In [11]:
featurenames = X.columns

---

### Cross-validate a linear regression predicting your target variable from the other variables

How does it perform?

In [12]:
linreg = LinearRegression()

lr_scores = cross_val_score(linreg, X, gradrate, cv=10)

linreg.fit(X, gradrate)

print lr_scores, np.mean(lr_scores)

[ 0.44921644  0.35875931  0.53650887  0.48549145  0.17445097  0.38775448
  0.17393223  0.43964213  0.61095627  0.3575135 ] 0.397422563045


In [13]:
for f, c in zip(featurenames, linreg.coef_):
    print f, c

private 3.38137582338
apps 0.00129841619653
accept -0.000696123912254
enroll 0.00215927836564
top10perc 0.0548964393801
top25perc 0.13512882248
f_undergrad -0.000471221055472
p_undergrad -0.00148364257626
outstate 0.00101744533787
room_board 0.001914341067
books -0.00222046570937
personal -0.00166353476834
phd 0.0872827138864
terminal -0.07470226732
s_f_ratio 0.0758222052443
perc_alumni 0.279334292031
expend -0.000456463007097


---

### Create a function that will iteratively predict your target from different train-test splits

This will be used to calculate the bias and the variance after this.

Your function should:

1. Accept a model, X predictor matrix/dataframe, y target variable, and a number of random splits to do training and testing on.
2. The output should be a dataframe that has as its first column the true values of y, and all the other columns will be corresponding predicted values of y when that row was in the testing set.
3. It will iterate through the number of splits
4. Create a variable that is the list of row numbers. Use this with `train_test_split` to get out randomized training rows and testing rows for each iteration.
5. Subset your X and y into training and testing
6. Train your model on the training X and training y
7. Predict values of y using the testing X
8. Add the predicted values of y to the dataframe tracking y predictions - the predicted y values should be insert in the correct row so that they match the true value of y in the first column. You can index using the test indices you got out of train_test_split to do this. (The rest of the rows that were part of the training set can be nan for that iteration).


In [14]:
def predict_from_samples(model, X, y, number_of_splits=100):
    
    yhat_tracker = pd.DataFrame({'ytrue':y})
    
    rowinds = range(X.shape[0])
    
    for i in range(number_of_splits):
        
        train_inds, test_inds = train_test_split(rowinds, test_size=0.33)
                
        Xtrain, Ytrain = X.iloc[train_inds, :], y[train_inds]
        Xtest, Ytest = X.iloc[test_inds, :], y[test_inds]
        
        model.fit(Xtrain, Ytrain)
        yhats = model.predict(Xtest)
        
        yhat_tracker['sample'+str(i+1)] = np.nan
        yhat_tracker.iloc[test_inds, -1] = yhats
        
    return yhat_tracker



---

### Create different predictor datasets

To see what happens to bias and variance as the predictors change, create a few versions of X that have different numbers of predictors in them.

For example, one could have all the other variables, and another one could be predicting only using private vs. public.

In [16]:
overfit_formula = '~ ('+' + '.join(featurenames)+')**3 -1'
X_overfit = patsy.dmatrix(overfit_formula, data=X, return_type='dataframe')

X_small = X[['private']]

---

### Use the predict function you wrote above to get the predicted values for each version of the data

Run each of your X through the function with the y target vector. As you recall the output of your function has the true values of y in a column and then predicted values of y in other columns for the different train-test splits

In [18]:
yhats_full = predict_from_samples(linreg, X, gradrate)
yhats_small = predict_from_samples(linreg, X_small, gradrate)
yhats_over = predict_from_samples(linreg, X_overfit, gradrate)

print X.shape, X_small.shape, X_overfit.shape
print yhats_full.shape, yhats_small.shape, yhats_over.shape

(777, 17) (777, 1) (777, 833)
(777, 101) (777, 101) (777, 101)


In [19]:
yhats_full.head(5)

,ytrue,sample1,sample2,sample3,sample4,sample5,sample6,sample7,sample8,sample9,...,sample91,sample92,sample93,sample94,sample95,sample96,sample97,sample98,sample99,sample100
0,60,NaN,NaN,56.113095,NaN,58.109048,57.222657,NaN,NaN,NaN,...,NaN,NaN,NaN,56.654950,58.453544,55.56493,58.787531,NaN,NaN,NaN
1,56,NaN,61.948475,60.047009,NaN,62.902856,NaN,67.602936,65.144263,60.547369,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65.325611,NaN,61.765065
2,54,67.383571,NaN,NaN,NaN,NaN,66.940037,NaN,NaN,67.581074,...,67.108044,67.834374,68.662840,66.588271,66.624881,NaN,68.372004,67.287530,NaN,68.107013
3,59,NaN,NaN,77.968398,76.649296,NaN,77.104564,NaN,NaN,NaN,...,79.962883,74.799189,NaN,NaN,NaN,NaN,76.773169,77.227944,NaN,NaN
4,15,NaN,NaN,54.105130,NaN,53.289631,52.189103,52.893361,NaN,NaN,...,52.014732,48.367786,50.102923,NaN,NaN,NaN,52.275562,NaN,52.404443,NaN


---

### Calculate bias and variance 

I've given you two functions below to calculate bias and variance if they are given the dataframe that has the first column as the true y values and the other column the predicted y values at each train/test split iteration.

You can use these to calculate the bias and variance of your different predictor variables. If you have more predictors variance of prediction should generally go up and bias goes down. Likewise, if you have few predictors variance should go down and bias goes up.

If you have an insanely bad model, they both might go up a lot!

In [20]:
def calculate_bias_sq(yhats_df):
    # Take out the true values of y that are in the first column:
    ytrue = yhats_df.iloc[:,0].values
    
    # Calculate the mean of the predictions, averaged across the columns.
    # So, all of the predictions for the true y at row 0 would be averaged together
    # and so on for all the rows.
    yhat_means = yhats_df.iloc[:,1:].mean(axis=1).values
    
    # Subtract the true value of y from the mean of the predicted values, and square it.
    elementwise_bias_sq = (yhat_means - ytrue)**2
    
    # Take the mean of those squared bias values (across all y)
    mean_bias_sq = np.mean(elementwise_bias_sq)
    return mean_bias_sq


def calculate_variance(yhats_df):
    # Calculate the mean of the predicted y's across the columns (mean of yhat for each row)
    yhats_means = yhats_df.iloc[:,1:].mean(axis=1)
    
    # subtract the mean of the yhats from the original yhat values (for each row)
    # and square the result. 
    yhats_devsq = yhats_df.iloc[:,1:].subtract(yhats_means, axis=0)**2
    
    # Take the mean of the squared deviations from the mean, then 
    # take the mean of those to get the overall variance across the y observations
    yhats_devsq_means = yhats_devsq.mean(axis=1).values
    return np.mean(yhats_devsq_means)


In [21]:
calculate_variance(yhats_full)

2.6099394624955208

In [22]:
print calculate_bias_sq(yhats_full)

169.578827015


In [23]:
calculate_variance(yhats_small)

0.26225975397414231

In [24]:
calculate_bias_sq(yhats_small)

262.49034543998511

In [25]:
calculate_variance(yhats_over)

4376059.8903146759

In [26]:
calculate_bias_sq(yhats_over)

122073.57701917981

---

### How does regularization affect bias and variance?

Use Lasso and/or Ridge on your dataset with all the predictor variables. You can feed the lasso or ridge model into the function you wrote earlier to get the predictions using regularization instead of just ordinary least squares regression.

How does using regularization affect bias and variance?